In [ ]:
WITH 
# Tablo 1 - tum kullanicilar 
all_users as 
(
SELECT DISTINCT user_pseudo_id as user_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
),
#Tablo 2- x kullanicisinin is_limited_tracking var mi
is_limited_tracking as 
(
SELECT distinct user_pseudo_id, device.is_limited_ad_tracking	
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
),
#Tablo 3 – x kullanicisi ard arda 3 gun session_start yapti mi
uc_gun_session_start as 
(
SELECT user_pseudo_id,
CASE 
WHEN max_consecuitive_days >=3 THEN True 
ELSE False
END as ucgun_ard_arda_session_start
FROM 
(
SELECT user_pseudo_id, MAX(consecuitive_days) as max_consecuitive_days
FROM (
SELECT user_pseudo_id, grp, COUNT(1) consecuitive_days
  FROM 
  (
SELECT user_pseudo_id, 
      COUNTIF(step > 1) OVER(PARTITION BY user_pseudo_id ORDER BY date) grp
    FROM
(
SELECT user_pseudo_id, date, 
        DATE_DIFF(date, LAG(date) OVER(PARTITION BY user_pseudo_id ORDER BY date), DAY) step 
      FROM 
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "session_start"
group by user_pseudo_id, date
))) GROUP BY user_pseudo_id, grp
)
GROUP BY user_pseudo_id
)),
# Tablo 4 - x kullanicisi ard arda 3 gun level_up yapti mi
uc_gun_level_up as 
(
SELECT user_pseudo_id,
CASE 
WHEN max_consecuitive_days >=3 THEN True 
ELSE False
END as ucgun_ard_arda_level_up
FROM 
(
SELECT user_pseudo_id, MAX(consecuitive_days) as max_consecuitive_days
FROM (
SELECT user_pseudo_id, grp, COUNT(1) consecuitive_days
  FROM 
  (
SELECT user_pseudo_id, 
      COUNTIF(step > 1) OVER(PARTITION BY user_pseudo_id ORDER BY date) grp
    FROM
(
SELECT user_pseudo_id, date, 
        DATE_DIFF(date, LAG(date) OVER(PARTITION BY user_pseudo_id ORDER BY date), DAY) step 
      FROM 
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "level_up"
group by user_pseudo_id, date
))) GROUP BY user_pseudo_id, grp
)
GROUP BY user_pseudo_id
)
),
# Tablo 5 - x kullanicisi ard arda 3 gun login yapti mi
uc_gun_login as 
(
SELECT user_pseudo_id,
CASE 
WHEN max_consecuitive_days >=3 THEN True 
ELSE False
END as ucgun_ard_arda_login
FROM 
(
SELECT user_pseudo_id, MAX(consecuitive_days) as max_consecuitive_days
FROM (
SELECT user_pseudo_id, grp, COUNT(1) consecuitive_days
  FROM 
  (
SELECT user_pseudo_id, 
      COUNTIF(step > 1) OVER(PARTITION BY user_pseudo_id ORDER BY date) grp
    FROM
(
SELECT user_pseudo_id, date, 
        DATE_DIFF(date, LAG(date) OVER(PARTITION BY user_pseudo_id ORDER BY date), DAY) step 
      FROM 
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "login"
group by user_pseudo_id, date
))) GROUP BY user_pseudo_id, grp
)
GROUP BY user_pseudo_id
)),
# Tablo 6 – uc gun ard arda event yapti mi ?
ucgun_ard_arda_event as 
(select user_pseudo_id,
CASE 
WHEN max_consecuitive_days >= 3 THEN True
ELSE False
END AS ucgun_ard_arda_event
from
(
SELECT user_pseudo_id, MAX(consecuitive_days) max_consecuitive_days
FROM (
SELECT user_pseudo_id, grp, COUNT(1) consecuitive_days
  FROM 
  (
SELECT user_pseudo_id, 
      COUNTIF(step > 1) OVER(PARTITION BY user_pseudo_id ORDER BY date) grp
    FROM
(
SELECT user_pseudo_id, date, 
        DATE_DIFF(date, LAG(date) OVER(PARTITION BY user_pseudo_id ORDER BY date), DAY) step 
      FROM 
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
group by user_pseudo_id, date
))) GROUP BY user_pseudo_id, grp
)
GROUP BY user_pseudo_id
)),
# Tablo 7 - X kullanicisi app_exception hatasi aldi mi?
has_app_exception as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "app_exception") then True 
ELSE False
end has_app_exception
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)
order by has_app_exception desc),
# Tablo 8 - X kullanicisi app_remove yapti mi?
app_removed_by_user as (

select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "app_remove") then True 
ELSE False
end as app_removed_by_user
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)
),
# Tablo 9 – x kullanicisi gunde ortalama kac event yapiyor ?
avg_daily_event as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_event
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 10 - X kullanicisi oyunu kac dk\saat\gun oynamis ?
kac_gun_saat_dk_second as (
SELECT user_pseudo_id,
TIMESTAMP_MICROS(min(event_timestamp)) as start_time, 
TIMESTAMP_MICROS(max(event_timestamp)) as end_time,
TIMESTAMP_DIFF(TIMESTAMP_MICROS(max(event_timestamp)), TIMESTAMP_MICROS(min(event_timestamp)), DAY) as active_day,
TIMESTAMP_DIFF(TIMESTAMP_MICROS(max(event_timestamp)), TIMESTAMP_MICROS(min(event_timestamp)), HOUR) as active_hour,
TIMESTAMP_DIFF(TIMESTAMP_MICROS(max(event_timestamp)), TIMESTAMP_MICROS(min(event_timestamp)), MINUTE) as active_minute,
TIMESTAMP_DIFF(TIMESTAMP_MICROS(max(event_timestamp)), TIMESTAMP_MICROS(min(event_timestamp)), SECOND) as active_seconds
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
group by user_pseudo_id
),
# Tablo 11 – x kullanicisinin cihaz kategorisi (mobile,tablet) ?
device_category as (
select distinct user_pseudo_id, device.category	as device_category
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
),
# Tablo 12 – x kullanicisinin cihaz marka ismi ?
mobile_brand_name as (
select distinct user_pseudo_id, device.mobile_brand_name as mobile_brand_name
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
),
# Tablo 13 – x kullanicisinin isletim sistemi ?
operating_system as (
select distinct user_pseudo_id, device.operating_system	as operating_system
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
),
# Tablo 14 – x kullanicisinin oyunu hangi marketten install yapti ?
max_appInfo_install_source as (
select  user_pseudo_id, max(app_info.install_source) as max_appInfo_install_source
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
Group By user_pseudo_id
),
# Tablo 15 – x kullanicisinin isletim sistemi versiyonu ?
max_operating_system_version as (
select user_pseudo_id, max(device.operating_system_version) as max_operating_system_version 
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
group by user_pseudo_id
),
# Tablo 16 – x kullanicisi hangi ulkeden max giris yapmis ?
country_max_event as (
select  user_pseudo_id, max(geo.country) as country_max_event
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
Group By user_pseudo_id
),
# Tablo 17 – x kullanicisi os_update (isletim sistemi) gunceleme yapti mi  ?
os_updated_by_user as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "os_update") then True 
ELSE False
end os_updated_by_user
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)
),
# Tablo 18 – x kullanicisi kampanyayi izlemeye basliyor mi (adCampaignWatchStart)  ?
ad_campaign_watch_start as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "adCampaignWatchStart") then True 
ELSE False
end ad_campaign_watch_start
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)
),
# Tablo 19 – x kullanicisi kampanyayi aciyor mu /click  yapiyor mu (adCampaignOpen)  ?
ad_campaign_open as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "adCampaignOpen") then True 
ELSE False
end ad_campaign_open
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)
),
# Tablo 20 – x kullanicisi kampanyayi sonuna kadar izledi mi  (adCampaignWatchComplete) ? 
ad_campaign_watch_complete as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "adCampaignWatchComplete") then True 
ELSE False
end ad_campaign_watch_complete
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)

),
# Tablo 21 – x kullanicisi uygulamayi guncelledi mi (app_update) ?
app_updated_by_user as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "app_update") then True 
ELSE False
end app_updated_by_user
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)

),
# Tablo 22 – x kullanicisi oyunun tutorialini bitir mi (tutorial_end)  ?
tutorial_end_by_user as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "tutorial_end") then True 
ELSE False
end tutorial_end_by_user
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)
),
# Tablo 23 – x kullanicisi game intro bitiriyor mu (gameIntroComplete)  ?
game_intro_completed_by_user as (
select user_pseudo_id,
case 
when user_pseudo_id in (select user_pseudo_id FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
where event_name = "gameIntroComplete") then True 
ELSE False
end game_intro_completed_by_user
from 
(
select user_pseudo_id
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata` 
Group by user_pseudo_id
)),
# Tablo 24 - x kullanicisi gunluk ortalama session_start yapiyor ?
avg_daily_session_start as 
(
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_session_start
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "session_start"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 25 - x kullanicisi gunluk ortalama level_up yapiyor ?
avg_daily_level_up as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_level_up
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "level_up"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 26 - x kullanicisi gunluk ortalama login yapiyor ?
avg_daily_login as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_login
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "login"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 27 - x kullanicisi gunluk ortalama kac adCampaignWatchComplete izliyor ?
avg_daily_adCampaignWatchComplete as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_adCampaignWatchComplete
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "adCampaignWatchComplete"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 28 - x kullanicisi gunluk ortalama kac defa workerUpgradeMethodExpensiveProduct ile yapiyor ?
avg_daily_workerUpgradeMethodExpensiveProduct as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_workerUpgradeMethodExpensiveProduct
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "workerUpgradeMethodExpensiveProduct"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 29 - x kullanicisi gunluk ortalama kac defa workerUpgradeAmount yapiyor ?
avg_daily_workerUpgradeAmount as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_workerUpgradeAmount
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "workerUpgradeAmount"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 30 - x kullanicisi gunluk ortalama kac defa workerUpgradeMethodLowestCost ile yapiyor ?
avg_daily_workerUpgradeMethodLowestCost as (
select user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_workerUpgradeMethodLowestCost
from
(
select user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
from `bootcamp-bigquery.KodluyoruzGameData.gamedata`
where event_name = "workerUpgradeMethodLowestCost"
group by user_pseudo_id, date
)
group by user_pseudo_id 
),
# Tablo 31 - x kullanicisi gunluk ortalama kac defa rocketAdWatchComplete ile yapiyor ?
avg_daily_rocketAdWatchComplete AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_rocketAdWatchComplete
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "rocketAdWatchComplete"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 32 - x kullanicisi gunluk ortalama kac defa error aliyor ?
avg_daily_error AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_error
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "error"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 33 - x kullanicisi gunluk ortalama kac defa unlockedWorker yapiyor ?
avg_daily_unlockedWorker AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_unlockedWorker
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "unlockedWorker"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 34 - x kullanicisi gunluk ortalama kac defa unlockedWorker yapiyor ?
avg_daily_earningRateBoost AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_earningRateBoost
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "earningRateBoost"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 35 - x kullanicisi gunluk ortalama kac defa rocketAdAppear reklamlari cikiyor ?
avg_daily_rocketAdAppear AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_rocketAdAppear
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "rocketAdAppear"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 36 - x kullanicisi gunluk ortalama kac ufoAdWatchComplete reklamlar sonuna kadar izliyor ?
avg_daily_ufoAdWatchComplete AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_ufoAdWatchComplete
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "ufoAdWatchComplete"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 37 - x kullanicisi gunluk ortalama kac defa unlockedFactory yapiyor ?
avg_daily_unlockedFactory AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_unlockedFactory
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "unlockedFactory"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 38 - x kullanicisi gunluk ortalama kac defa productScreenClick tiklamasi yapiyor ?
avg_daily_productScreenClick AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_productScreenClick
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "productScreenClick"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 39 - x kullanicisi gunluk ortalama kac defa ad_reward odul kazaniyor ?
avg_daily_ad_reward AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_ad_reward
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "ad_reward"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
),
# Tablo 40 - x kullanicisi gunluk ortalama kac defa workerUpgradeUpgradeMultiplierMax yapiyor ?
avg_daily_workerUpgradeUpgradeMultiplierMax AS (
SELECT user_pseudo_id, (sum(event_count) / count(date)) as avg_daily_workerUpgradeUpgradeMultiplierMax
FROM
(
SELECT user_pseudo_id, cast(TIMESTAMP_MICROS(event_timestamp) as DATE) AS date, count(event_name) as event_count
FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
WHERE event_name = "workerUpgradeUpgradeMultiplierMax"
GROUP BY user_pseudo_id, date
)
GROUP BY user_pseudo_id 
), 
# Tablo 41 - x kullanicisinin ortalama session bazli aktivitileri ?
session_based_activity as
(
SELECT
user_pseudo_id, 
COUNTIF(event_name = "level_up" AND (
SELECT value.string_value = "Worker" FROM UNNEST(event_params)
WHERE key = "type")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params)WHERE key = "ga_session_number"))) 
AS avg_worker_session,

COUNTIF(event_name = "level_up" AND (
SELECT value.string_value = "Factory" FROM UNNEST(event_params)
WHERE key = "type")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_factory_session,

COUNTIF(event_name = "level_up" AND (
SELECT value.string_value = "Transporter" FROM UNNEST(event_params)
WHERE key = "type")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_transporter_session,
COUNTIF(event_name = "level_up" AND (
SELECT value.string_value = "BaseStation" FROM UNNEST(event_params)
WHERE key = "type")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_baseStation_session,

COUNTIF(event_name = "ad_impression_ga" AND (
SELECT value.string_value = "RewardedVideo" FROM UNNEST(event_params)
WHERE key = "type")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_RewardedVideo_session,

SUM (( SELECT value.int_value FROM UNNEST(event_params)
WHERE key = "upgradeAmount")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_upgradeAmount_session,

SUM (( SELECT value.double_value FROM UNNEST(event_params)
WHERE key = "totalElapsedTime")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_totalElapsedTime_session,

SUM (( SELECT value.double_value FROM UNNEST(event_params)
WHERE key = "earningRateValue")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_earningRateValue_session,

SUM (( SELECT value.int_value FROM UNNEST(event_params)
WHERE key = "freeride")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_freeride_session,

SUM((SELECT value.int_value FROM UNNEST(event_params)
where event_name = "user_engagement" AND key = "engagement_time_msec" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_uengagement_time_msec_session,

SUM((SELECT value.int_value FROM UNNEST(event_params)
where event_name = "screen_view" AND key = "engagement_time_msec" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_sengagement_time_msec_session,

SUM (( SELECT value.int_value FROM UNNEST(event_params)
WHERE key = "reward_value")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_reward_value_session,

SUM((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "workerUpgradeUpgradeMultiplierMax" AND key = "upgradeMultiplierValue" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_maxupgradeMultiplierValue_session,

COUNT((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "workerUpgradeUpgradeMultiplierx1" AND key = "upgradeMultiplierValue" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_x1upgradeMultiplierValue_session,

COUNT((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "workerUpgradeUpgradeMultiplierx10" AND key = "upgradeMultiplierValue" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_x10upgradeMultiplierValue_session,

COUNT((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "workerUpgradeUpgradeMultiplierx50" AND key = "upgradeMultiplierValue" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_x50upgradeMultiplierValue_session,

SUM((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "adCampaignOpen" AND key = "activeAdCampaignAmount" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_OpenactiveAdCampaignAmount_session,

SUM((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "adCampaignExit" AND key = "activeAdCampaignAmount" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_ExitactiveAdCampaignAmount_session,

SUM((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "adCampaignWatchStart" AND key = "activeAdCampaignAmount" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_StartactiveAdCampaignAmount_session,

SUM((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "adCampaignWatchNoncomplete" AND key = "activeAdCampaignAmount" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_NoncompleteactiveAdCampaignAmount_session,

SUM((SELECT value.double_value FROM UNNEST(event_params)
where event_name = "adCampaignWatchComplete" AND key = "activeAdCampaignAmount" ))/ (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_CompleteactiveAdCampaignAmount_session,

COUNTIF(event_name = "error")/ (MAX(( SELECT value.int_value FROM UNNEST(event_params)WHERE key = "ga_session_number"))) 
AS avg_error_session,

COUNTIF(event_name = "level_up")/ (MAX(( SELECT value.int_value FROM UNNEST(event_params)WHERE key = "ga_session_number"))) 
AS avg_level_up_session,

COUNTIF(event_name = "unlockedWorker")/ (MAX(( SELECT value.int_value FROM UNNEST(event_params)WHERE key = "ga_session_number"))) 
AS avg_unlockedWorker_session,

COUNTIF(event_name = "ad_impression_ga" AND (
SELECT value.string_value = "AD_CAMPAIGN_X2_CASH" FROM UNNEST(event_params)
WHERE key = "placementName")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params)WHERE key = "ga_session_number"))) 
AS avg_AD_CAMPAIGN_X2_CASH_session,

COUNTIF(event_name = "ad_impression_ga" AND (
SELECT value.string_value = "IDLE_CASH_AD" FROM UNNEST(event_params)
WHERE key = "placementName")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_IDLE_CASH_AD_session,

COUNTIF(event_name = "level_up" AND (
SELECT value.string_value = "ROCKET_AD" FROM UNNEST(event_params)
WHERE key = "placementName")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_ROCKET_AD_session,

COUNTIF(event_name = "ad_impression_ga" AND (
SELECT value.string_value = "UFO_AD" FROM UNNEST(event_params)
WHERE key = "placementName")) / (MAX(( SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_number")))
AS avg_UFO_AD_session

FROM `bootcamp-bigquery.KodluyoruzGameData.gamedata`
group by user_pseudo_id
)


SELECT 
# tablo 1
all_users.user_id,
# tablo 2
is_limited_tracking.is_limited_ad_tracking,
# tablo 3
uc_gun_session_start.ucgun_ard_arda_session_start,
# tablo 4
uc_gun_level_up.ucgun_ard_arda_level_up,
# tablo 5
uc_gun_login.ucgun_ard_arda_login,
# tablo 6
ucgun_ard_arda_event.ucgun_ard_arda_event,
# tablo 7
has_app_exception.has_app_exception,
# tablo 8
app_removed_by_user.app_removed_by_user,
# tablo 9
avg_daily_event.avg_daily_event,
# tablo 10
kac_gun_saat_dk_second.start_time,
kac_gun_saat_dk_second.end_time,
kac_gun_saat_dk_second.active_day,
kac_gun_saat_dk_second.active_hour,
kac_gun_saat_dk_second.active_minute,
kac_gun_saat_dk_second.active_seconds,
# tablo 11
device_category.device_category,
# tablo 12
mobile_brand_name.mobile_brand_name,
# tablo 13
operating_system.operating_system,
# tablo 14
max_appInfo_install_source.max_appInfo_install_source,
# tablo 15
max_operating_system_version.max_operating_system_version,
# tablo 16
country_max_event.country_max_event,
# tablo 17
os_updated_by_user.os_updated_by_user,
# tablo 18
ad_campaign_watch_start.ad_campaign_watch_start,
# tablo 19
ad_campaign_open.ad_campaign_open,
# tablo 20
ad_campaign_watch_complete.ad_campaign_watch_complete,
# tablo 21
app_updated_by_user.app_updated_by_user,
# tablo 22
tutorial_end_by_user.tutorial_end_by_user,
# tablo 23
game_intro_completed_by_user.game_intro_completed_by_user,
# tablo 24
case 
when avg_daily_session_start.avg_daily_session_start is not null then avg_daily_session_start.avg_daily_session_start
else 0
end as avg_daily_session_start,
# tablo 25
case 
when avg_daily_level_up.avg_daily_level_up is not null then avg_daily_level_up.avg_daily_level_up
else 0
end as avg_daily_level_up,
# tablo 26
case 
when avg_daily_login.avg_daily_login is not null then avg_daily_login.avg_daily_login
else 0
end as avg_daily_login,
# tablo 27
case 
when avg_daily_adCampaignWatchComplete.avg_daily_adCampaignWatchComplete is not null then avg_daily_adCampaignWatchComplete.avg_daily_adCampaignWatchComplete
else 0
end as avg_daily_adCampaignWatchComplete,
# tablo 28
case 
when avg_daily_workerUpgradeMethodExpensiveProduct.avg_daily_workerUpgradeMethodExpensiveProduct is not null then avg_daily_workerUpgradeMethodExpensiveProduct.avg_daily_workerUpgradeMethodExpensiveProduct
else 0
end as avg_daily_workerUpgradeMethodExpensiveProduct,
# tablo 29
case 
when avg_daily_workerUpgradeAmount.avg_daily_workerUpgradeAmount is not null then avg_daily_workerUpgradeAmount.avg_daily_workerUpgradeAmount
else 0
end as avg_daily_workerUpgradeAmount,
# tablo 30
case 
when avg_daily_workerUpgradeMethodLowestCost.avg_daily_workerUpgradeMethodLowestCost is not null then avg_daily_workerUpgradeMethodLowestCost.avg_daily_workerUpgradeMethodLowestCost
else 0
end as avg_daily_workerUpgradeMethodLowestCost,
# tablo 31
case 
when avg_daily_rocketAdWatchComplete.avg_daily_rocketAdWatchComplete is not null then avg_daily_rocketAdWatchComplete.avg_daily_rocketAdWatchComplete
else 0
end as avg_daily_rocketAdWatchComplete,
# tablo 32
case 
when avg_daily_error.avg_daily_error is not null then avg_daily_error.avg_daily_error
else 0
end as avg_daily_error,
# tablo 33
case 
when avg_daily_unlockedWorker.avg_daily_unlockedWorker is not null then avg_daily_unlockedWorker.avg_daily_unlockedWorker
else 0
end as avg_daily_unlockedWorker,
# tablo 34
case 
when avg_daily_earningRateBoost.avg_daily_earningRateBoost is not null then avg_daily_earningRateBoost.avg_daily_earningRateBoost
else 0
end as avg_daily_earningRateBoost,
# tablo 35
case 
when avg_daily_rocketAdAppear.avg_daily_rocketAdAppear is not null then avg_daily_rocketAdAppear.avg_daily_rocketAdAppear
else 0
end as avg_daily_rocketAdAppear,
# tablo 36
case 
when avg_daily_ufoAdWatchComplete.avg_daily_ufoAdWatchComplete is not null then avg_daily_ufoAdWatchComplete.avg_daily_ufoAdWatchComplete
else 0
end as avg_daily_ufoAdWatchComplete,
# tablo 37
case 
when avg_daily_unlockedFactory.avg_daily_unlockedFactory is not null then avg_daily_unlockedFactory.avg_daily_unlockedFactory
else 0
end as avg_daily_unlockedFactory,
# tablo 38
case 
when avg_daily_productScreenClick.avg_daily_productScreenClick is not null then avg_daily_productScreenClick.avg_daily_productScreenClick
else 0
end as avg_daily_productScreenClick,
# tablo 39
case 
when avg_daily_ad_reward.avg_daily_ad_reward is not null then avg_daily_ad_reward.avg_daily_ad_reward
else 0
end as avg_daily_ad_reward,
# tablo 40
case 
when avg_daily_workerUpgradeUpgradeMultiplierMax.avg_daily_workerUpgradeUpgradeMultiplierMax is not null then avg_daily_workerUpgradeUpgradeMultiplierMax.avg_daily_workerUpgradeUpgradeMultiplierMax
else 0
end as avg_daily_workerUpgradeUpgradeMultiplierMax,
# tablo 41
session_based_activity.avg_worker_session,
session_based_activity.avg_factory_session,
session_based_activity.avg_transporter_session,
session_based_activity.avg_baseStation_session,
session_based_activity.avg_RewardedVideo_session,
session_based_activity.avg_upgradeAmount_session,
session_based_activity.avg_totalElapsedTime_session,
session_based_activity.avg_earningRateValue_session,
session_based_activity.avg_freeride_session,
session_based_activity.avg_uengagement_time_msec_session,
session_based_activity.avg_sengagement_time_msec_session,
session_based_activity.avg_reward_value_session,
session_based_activity.avg_maxupgradeMultiplierValue_session,
session_based_activity.avg_x1upgradeMultiplierValue_session,
session_based_activity.avg_x10upgradeMultiplierValue_session,
session_based_activity.avg_x50upgradeMultiplierValue_session,
session_based_activity.avg_OpenactiveAdCampaignAmount_session,
session_based_activity.avg_ExitactiveAdCampaignAmount_session,
session_based_activity.avg_StartactiveAdCampaignAmount_session,
session_based_activity.avg_NoncompleteactiveAdCampaignAmount_session,
session_based_activity.avg_CompleteactiveAdCampaignAmount_session,
session_based_activity.avg_error_session,
session_based_activity.avg_level_up_session,
session_based_activity.avg_unlockedWorker_session,
session_based_activity.avg_AD_CAMPAIGN_X2_CASH_session,
session_based_activity.avg_IDLE_CASH_AD_session,
session_based_activity.avg_ROCKET_AD_session,
session_based_activity.avg_UFO_AD_session


FROM all_users
FULL OUTER JOIN is_limited_tracking on all_users.user_id = is_limited_tracking.user_pseudo_id
FULL OUTER JOIN uc_gun_session_start on all_users.user_id = uc_gun_session_start.user_pseudo_id
FULL OUTER JOIN uc_gun_level_up on all_users.user_id = uc_gun_level_up.user_pseudo_id
FULL OUTER JOIN uc_gun_login on all_users.user_id = uc_gun_login.user_pseudo_id
FULL OUTER JOIN ucgun_ard_arda_event on all_users.user_id = ucgun_ard_arda_event.user_pseudo_id
FULL OUTER JOIN has_app_exception on all_users.user_id = has_app_exception.user_pseudo_id
FULL OUTER JOIN app_removed_by_user on all_users.user_id = app_removed_by_user.user_pseudo_id
FULL OUTER JOIN avg_daily_event on all_users.user_id = avg_daily_event.user_pseudo_id
FULL OUTER JOIN kac_gun_saat_dk_second on all_users.user_id = kac_gun_saat_dk_second.user_pseudo_id
FULL OUTER JOIN device_category on all_users.user_id = device_category.user_pseudo_id
FULL OUTER JOIN mobile_brand_name on all_users.user_id = mobile_brand_name.user_pseudo_id
FULL OUTER JOIN operating_system on all_users.user_id = operating_system.user_pseudo_id
FULL OUTER JOIN max_appInfo_install_source on all_users.user_id = max_appInfo_install_source.user_pseudo_id
FULL OUTER JOIN max_operating_system_version on all_users.user_id = max_operating_system_version.user_pseudo_id
FULL OUTER JOIN country_max_event on all_users.user_id = country_max_event.user_pseudo_id
FULL OUTER JOIN os_updated_by_user on all_users.user_id = os_updated_by_user.user_pseudo_id
FULL OUTER JOIN ad_campaign_watch_start on all_users.user_id = ad_campaign_watch_start.user_pseudo_id
FULL OUTER JOIN ad_campaign_open on all_users.user_id = ad_campaign_open.user_pseudo_id
FULL OUTER JOIN ad_campaign_watch_complete on all_users.user_id = ad_campaign_watch_complete.user_pseudo_id
FULL OUTER JOIN app_updated_by_user on all_users.user_id = app_updated_by_user.user_pseudo_id
FULL OUTER JOIN tutorial_end_by_user on all_users.user_id = tutorial_end_by_user.user_pseudo_id
FULL OUTER JOIN game_intro_completed_by_user on all_users.user_id = game_intro_completed_by_user.user_pseudo_id
FULL OUTER JOIN avg_daily_session_start on all_users.user_id = avg_daily_session_start.user_pseudo_id
FULL OUTER JOIN avg_daily_level_up on all_users.user_id = avg_daily_level_up.user_pseudo_id
FULL OUTER JOIN avg_daily_login on all_users.user_id = avg_daily_login.user_pseudo_id
FULL OUTER JOIN avg_daily_adCampaignWatchComplete on all_users.user_id = avg_daily_adCampaignWatchComplete.user_pseudo_id
FULL OUTER JOIN avg_daily_workerUpgradeMethodExpensiveProduct on all_users.user_id = avg_daily_workerUpgradeMethodExpensiveProduct.user_pseudo_id
FULL OUTER JOIN avg_daily_workerUpgradeAmount on all_users.user_id = avg_daily_workerUpgradeAmount.user_pseudo_id
FULL OUTER JOIN avg_daily_workerUpgradeMethodLowestCost on all_users.user_id = avg_daily_workerUpgradeMethodLowestCost.user_pseudo_id
FULL OUTER JOIN avg_daily_rocketAdWatchComplete on all_users.user_id = avg_daily_rocketAdWatchComplete.user_pseudo_id
FULL OUTER JOIN avg_daily_error on all_users.user_id = avg_daily_error.user_pseudo_id
FULL OUTER JOIN avg_daily_unlockedWorker on all_users.user_id = avg_daily_unlockedWorker.user_pseudo_id
FULL OUTER JOIN avg_daily_earningRateBoost on all_users.user_id = avg_daily_earningRateBoost.user_pseudo_id
FULL OUTER JOIN avg_daily_rocketAdAppear on all_users.user_id = avg_daily_rocketAdAppear.user_pseudo_id
FULL OUTER JOIN avg_daily_ufoAdWatchComplete on all_users.user_id = avg_daily_ufoAdWatchComplete.user_pseudo_id
FULL OUTER JOIN avg_daily_unlockedFactory on all_users.user_id = avg_daily_unlockedFactory.user_pseudo_id
FULL OUTER JOIN avg_daily_productScreenClick on all_users.user_id = avg_daily_productScreenClick.user_pseudo_id
FULL OUTER JOIN avg_daily_ad_reward on all_users.user_id = avg_daily_ad_reward.user_pseudo_id
FULL OUTER JOIN avg_daily_workerUpgradeUpgradeMultiplierMax on all_users.user_id = avg_daily_workerUpgradeUpgradeMultiplierMax.user_pseudo_id
FULL OUTER JOIN session_based_activity on all_users.user_id = session_based_activity.user_pseudo_id
